# Word embeddings

단어 임베딩(Word Embedding)이란 텍스트를 구성하는 하나의 단어를 수치화하는 방법의 일종이다.

텍스트에 나타난 단어를 수만개가 될수 있는데 이것을 one-hot encoding 방식으로 처리하는 것은 매우 비효율적이다. 단어를 one-hot vector로 나타내면 대부분 0 값이 된다. hidden layer에서 one-hot input vector와 연산이 대부분 0이 된다. 

이런 비효율성을 개선하기 위한 것이 **embeddings** 이다. Embedding layer는 앞서 본 fully connected layer와 같은 형태이다. embedding layer의 weights를 embedding weights라고 한다. 

![img](../assets/lookup_matrix.png)
특정 단어와 맵핑되는 정수를 인덱스로 가지는 테이블로부터 임베딩 벡터 값을 가져오는 룩업 테이블이라고 볼 수 있다. 그리고 이 테이블은 단어 집합의 크기만큼의 행을 가지므로 모든 단어는 고유한 임베딩 벡터를 가진다. 

words를 integers 로 변환하게 된다. 단어 great은 정수 인코딩 과정에서 1918의 정수로 인코딩이 되었고 그에 따라 단어 집합의 크기만큼의 행을 가지는 테이블에서 인덱스 1918번에 위치한 행을 단어 great의 임베딩 벡터로 사용한다. 이것을 **embedding lookup** 라고 하고 hidden units의 갯수가 **embedding dimension**이 된다.
 
embedding lookup table은 단순한 weight matrix이고 embedding layer는 hidden layer이다. 

---

# Pre-trained Word Embedding

**사전 훈련된 워드 임베딩(pre-trained word embedding)**


위키피디아 등과 같은 방대한 코퍼스를 가지고 사전에 학습된 embedding vector를 사용할 수 있다.

In [1]:
import torch
import torchtext

In [2]:
# 현재 위치에 .vector_cache 디렉토리를 생성하여 GloVe 파일을 다운로드한다.
# 이미 다운로드 되어 있으면 다시 다운로드 하지 않고 읽어들인다.
glove = torchtext.vocab.GloVe(cache='data',
                              name="6B", # trained on Wikipedia 2014 corpus
                              dim=50)   # embedding size = 50

학습된 GloVe의 embedding vector값을 확인할 수 있다.

In [3]:
glove['cat']

tensor([ 0.4528, -0.5011, -0.5371, -0.0157,  0.2219,  0.5460, -0.6730, -0.6891,
         0.6349, -0.1973,  0.3368,  0.7735,  0.9009,  0.3849,  0.3837,  0.2657,
        -0.0806,  0.6109, -1.2894, -0.2231, -0.6158,  0.2170,  0.3561,  0.4450,
         0.6089, -1.1633, -1.1579,  0.3612,  0.1047, -0.7832,  1.4352,  0.1863,
        -0.2611,  0.8328, -0.2312,  0.3248,  0.1449, -0.4455,  0.3350, -0.9595,
        -0.0975,  0.4814, -0.4335,  0.6945,  0.9104, -0.2817,  0.4164, -1.2609,
         0.7128,  0.2378])

### Measuring Distance

각 단어의 vector간 유사도를 측정할 수 있다.

* Euclidean distance
* Cosine Similarity

In [4]:
# Euclidean distance
x = glove['cat']
y = glove['dog']
torch.norm(y - x)

tensor(1.8846)

cosine similarity를 사용하여 우리가 선택한 단어와 가장 유사한 단어를 출력한다.

<img src="../assets/two_vectors.png" width="300">
$$
\mathrm{similarity} = \cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|}
$$


validation words를  vectors $\vec{a}$ 로 encoding한다. 그리고 validation words 와 embedding table의 단어들에 대한 유사도를 확인한다. 이를 통해 embedding table 내에 학습된 단어들이 얼마나 잘 유사한 단어끼리 grouping 되어 있는지 확인할 수 있다.

In [5]:
x = glove['cat']
y = glove['dog']
torch.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0))

tensor([0.9218])

### Word Similarity
전체 어휘를 통해 임베딩 공간의 한 지점에 가장 가까운 단어를 찾을 수 있다. 예를 들어 "cat"와 같은 다른 단어와 가장 가까운 단어를 찾을 수 있다.

In [6]:
def print_closest_words(v, n=5):
    dists = torch.norm(glove.vectors - v, dim=1)     # compute distances to all words
    lst = sorted(enumerate(dists.numpy()), key=lambda x: x[1]) # sort by distance
    for idx, difference in lst[1:n+1]:         # take the top n
        print(glove.itos[idx], difference)

In [7]:
print_closest_words(glove['cat'])

dog 1.8846031
rabbit 2.4572797
monkey 2.8102052
cats 2.8972247
rat 2.9455352


In [8]:
print_closest_words(glove['nurse'])

doctor 3.1274529
dentist 3.1306612
nurses 3.26872
pediatrician 3.3212206
counselor 3.3987114


### Analogies - 유추

임베딩 공간의 방향이 의미가 있다. GloVe 벡터의 구조는 다음과 같은 특정 단어들간의 관계를 연산을 통해 확인할 수 있다.

```
king − man + woman ≈ queen
```

In [9]:
print_closest_words(glove['king'] - glove['man'] + glove['woman'])

queen 2.8391209
prince 3.6610038
elizabeth 3.7152522
daughter 3.8317878
widow 3.8493774


In [10]:
print_closest_words(glove['programmer'] - glove['bad'] + glove['good'])

versatile 4.381561
creative 4.5690007
entrepreneur 4.6343737
enables 4.7177725
intelligent 4.7349973


In [11]:
print_closest_words(glove['programmer'] - glove['good'] + glove['bad'])

hacker 3.8383653
glitch 4.003873
originator 4.041952
hack 4.047719
serial 4.2250676
